<a href="https://colab.research.google.com/github/aml7hawaiiedu/MoraLab/blob/main/download_and_read_papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## function to download and analyse papers from google scholar
- This function will do a google scholar search based on a query (food + hazard)
- It will attempt to download the first 20 results (mostly through scihub)
- Then ChatPDF will check through each pdf and answer your question and find quotes within each pdf

You need
1. To install PyPaperBot (you can do this in anaconda terminal by typing *pip install PyPaperBot*
2. A chatPDF API Key. you can get this by making a chatPDF user and clicking *My Account* on the ChatPDF API website https://www.chatpdf.com/docs/api/backend

In [ ]:
import requests
import glob
import PyPaperBot
from PyPaperBot import __main__ as p
import os

In [ ]:
# FUNCTION to download and review the papers

def download_and_read(food, hazard, API_Key, scholar_pages=[1,2], scholar_results=20, question='default', skip_if_folder_exists = True):

    ## ----------------------------------- ###
    # specify download folder
    folder_name = f'{food}_{hazard}'
    dwn_dir = os.path.join(os.getcwd(), folder_name)
    if os.path.exists(dwn_dir) and skip_if_folder_exists:
        pdfs = glob.glob(os.path.join(dwn_dir, '*.pdf'))
        print('found %d papers in the folder "%s"' %(len(pdfs), folder_name))
    else: # not os.path.exists(dwn_dir)
        os.mkdir(dwn_dir)
        # make query:
        query = f'{food}+{hazard}'
        #download papers from google scholar
        p.start(query=query, scholar_pages=scholar_pages, scholar_results=scholar_results, dwn_dir=dwn_dir, proxy=[])

        pdfs = glob.glob(os.path.join(dwn_dir, '*.pdf'))
        print('downloaded %d of %d papers' %(len(pdfs), scholar_results))

    ## ----------------------------------- ###
    # load papers into chatpdf
    for file in pdfs:
        print(f'loading file "{os.path.basename(file)}" ')
        files = [('file', ('file', open(file, 'rb'), 'application/octet-stream'))]
        headers = {'x-api-key': API_Key}
        response = requests.post('https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)
        if response.status_code == 200:
            sID = response.json()['sourceId']
            #print('Source ID:', response.json()['sourceId'])
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
        headers = {
            'x-api-key': API_Key,
            "Content-Type": "application/json",
        }

        ##### ----------------------------------- ###
        # 'read' papers
        print('analyzing file...')
        if question == 'default':
            question = f'How does {hazard} impact {food}? can you provide a quote from the text about this?'

        data = {
            'sourceId': sID,
            'messages': [
                {
                    'role': "user",
                    'content': question,
                }
            ]
        }
        response = requests.post(
            'https://api.chatpdf.com/v1/chats/message', headers=headers, json=data)

        if response.status_code == 200:
            print('result:', response.json()['content'])
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
        print("\n")


### Download and "read" the papers
- Change food and hazard to any combination and input your API_Key
- If you want to change any settings of numbers of pages and number of files to download
- You can also specify a question to ask, otherwise it will ask "How does {hazard} impact {food}? can you provide a quote from the text about this?"

In [ ]:
food = 'wheat'
hazard = 'drought'
API_Key = 'sec_XXXXXXXXXXXXXXXXXXX' #make a user and get api key from https://www.chatpdf.com/docs/api/backend
scholar_pages = [1,2]
scholar_results = 20
question = 'default' ## Specify a question in quotes or use the default: "How does {hazard} impact {food}? can you provide a quote from the text about this?"
skip_if_folder_exists = True ## dummy variable, if you have the data downloaded already and dont want to re-download it. defualt=True

## Run function
download_and_read(food, hazard, API_Key, scholar_pages=[1,2], scholar_results=20, question='default', skip_if_folder_exists = True)


found 20 papers in the folder "wheat_drought"
loading file "An EPIC model-based wheat drought risk assessment using new climate scenarios in China.pdf" 
analyzing file...
result: According to page 3, "Global agriculture is facing unprecedented challenges and risks. According to the latest Intergovernmental Panel on Climate Change (IPCC) assessment report, climate warming is beyond doubt (IPCC 2013). This will lead to an increased degrees of drought (Trenberth et al. 2013), thereby negatively impacting crop yields."


loading file "Genetic and genomic tools to improve drought tolerance in wheat.pdf" 
analyzing file...
result: According to page 6, "Wheat production relies on rainfall during the growing season when precipitation decreases with the rise of temperature during spring. The availability of water is cyclic with a succession of precipitation and drought periods from anthesis to the grain-filling stages. The drought is combined with a cyclic heat stress between rainfall events."
